In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import time

In [6]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [7]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [8]:
#! pip install fundamentos

In [9]:
#!pip install -q yfinance

In [10]:
import fundamentos as ftos

In [11]:
from pandas_datareader import data
import plotly.express as px
import yfinance as yf

# 1) clean the date quote data set

In [12]:
date_quote= pd.read_csv("date_quote.csv", sep=";", encoding = 'latin_1')

In [13]:
# Adjust date

In [14]:
date_quote['date_month']=date_quote['date'].str[3:5]
date_quote['date_year']=date_quote['date'].str[6:10]
date_quote['date']=date_quote['date'].str[6:10]+date_quote['date'].str[3:5]+date_quote['date'].str[0:2]

In [15]:
date_quote_clean=date_quote[['date', 'company' , 'date_month', 'date_year', 'close' ]]

In [16]:
date_quote_clean = date_quote_clean.reset_index()

In [17]:
date_quote_clean=date_quote_clean.drop(columns=['index'])

In [18]:
date_quote_clean['date_month']=date_quote_clean['date_month'].astype(int)
date_quote_clean['date_year']=date_quote_clean['date_year'].astype(int)

In [19]:
# add 
date_quote_manual= pd.read_csv("date_quote_manual.csv", sep=";", encoding = 'latin_1')

In [20]:
date_quote_end=pd.concat([date_quote_clean, date_quote_manual])

In [21]:
date_quote_end['date']=date_quote_end['date'].astype(int)

# 2) Contribution

In [22]:
date_quote_end=date_quote_end.sort_values(by=['company', 'date'], ascending=False)

In [23]:
#dropping ALL duplicate values
date_quote_end_1=date_quote_end.drop_duplicates(subset=["company", 'date_month','date_year'], keep = 'first').reset_index()

In [24]:
def create_contributions(df):
    df['contributions']=0
    contributions=0
    for year in range(2015,2022):
        for month in range(3, 13, 3):
            contributions=contributions+1
            # Condition
            conditions = [
                (df['date_year'] == year) & (df['date_month'] == month)]
            choices = [contributions]
            df['contributions'] = np.select(conditions, choices, default=df['contributions'])
    return df

In [25]:
date_quote_end_2=create_contributions(date_quote_end_1) 

In [26]:
date_quote_end_2.to_csv('date_quote_end_2.csv')

# 3) Create EPS (earning per share)

In [27]:
df_results= pd.read_csv("dateset_fundamentalist.csv", sep=";", encoding = 'latin_1')

In [28]:
df_results['eps']=df_results['eps'].astype('float')

In [29]:
# create the quarter

In [30]:
date_quote_filter=date_quote_end.sort_values(by='date', ascending=False)

In [31]:
date_quote_filter=date_quote_filter.drop_duplicates(subset=['company', 'date_month', 'date_year'], keep='first')

In [32]:
date_quote_filter=date_quote_filter[['company', 'date_month', 'date_year', 'close']]

In [33]:
df_results_price=pd.merge(df_results,date_quote_filter,how='left', 
                          left_on=['company','date_year','date_month'],right_on=['company','date_year','date_month'])

In [34]:
#  MERGE indicators with price. 
# Keep the companies we found the price

In [35]:
df_results_price_2=df_results_price[df_results_price['close'].notna()].reset_index()

In [36]:
df_results_price_2['pe']=df_results_price_2['close']/df_results_price_2['eps']
df_results_price_2['pe'] = df_results_price_2['eps'].apply(lambda x: 0 if x == 0 else x )

In [37]:
# delete duplicated companies (eg CGRA3 and CGRA4)
df_results_price_2=df_results_price_2.sort_values(by=['company'])
df_results_price_clean=df_results_price_2.drop_duplicates(subset=['name', 'date', ], keep='first')

# 4) Magic Formula Of Joel

In [38]:
def scoring(df, metric, ascend):
    df=df.sort_values(by=[metric], ascending=ascend).reset_index()
    df=df.drop(columns=['index'])
    var_score='score_'+metric
    df[var_score]=df.index
    return df

In [39]:
def formula_joel():
    date_joel= pd.DataFrame()
    for year in range(2015,2022):
        for month in range(3, 13, 3):
            df_selec=df_results_price_clean[(df_results_price_clean['date_year']==year) & 
                                            (df_results_price_clean['date_month']==month)]
            # we select only positive net_profit
            df_selec=df_selec[df_selec['net_profit_12m']>0]
            #score_pe
            df_selec=scoring(df=df_selec, metric='pe', ascend=False)
            # score_roe
            df_selec=scoring(df=df_selec, metric='roe', ascend=True)
            # score_joel
            df_selec['score_joel']=df_selec['score_roe']*df_selec['score_pe']
            df_selec=df_selec.sort_values(by=['score_joel'], ascending=False).reset_index()
            ##
            df_selec=df_selec.drop(columns=['index','level_0' ])
            df_selec=scoring(df=df_selec, metric='score_joel', ascend=False)
            df_selec=df_selec.rename(columns={'score_score_joel':'chosen'})
            date_joel = pd.concat([date_joel, df_selec])
    return date_joel 

In [40]:
df_joel=formula_joel()

In [41]:
df_joel_1=create_contributions(df_joel) 

## Calcute the profit 

In [42]:
date_quote_end_3=date_quote_end_2.rename(columns={'close':'close_end'})
date_quote_end_3['contributions']=date_quote_end_3['contributions']-1
date_quote_end_3=date_quote_end_3[['company', 'close_end','contributions']]

## Joel all the cicle

In [43]:
joel_performance = pd.DataFrame({'contributions': range(1,26), 'profit_total':range(1,26)})
date_detail_joel= pd.DataFrame()

investiment_month=10000
investiment=0

In [44]:
for contributions in range(26):
    contributions=contributions+1

    investiment=investiment+investiment_month
    investiment_per_share=investiment/10

    df_joel_aux=df_joel[(df_joel['contributions']==contributions)]
    df_joel_aux_1=pd.merge(df_joel_aux, date_quote_end_3, how='left', left_on=['company', 'contributions'], 
                           right_on=['company','contributions'])

    df_joel_aux_1['no_shares']=investiment_per_share/df_joel_aux_1['close']
    df_joel_aux_1['profit']=df_joel_aux_1['no_shares']*(df_joel_aux_1['close_end'])
    
    # selecet the 10 TOP companies 
    top_selected=df_joel_aux_1[df_joel_aux_1['chosen'].isin([0,1,2,3,4,5,6,7,8,9])]
    date_detail_joel= pd.concat([date_detail_joel, top_selected])
    investiment=top_selected['profit'].sum()
    
        # Condition
    conditions = [
        joel_performance['contributions']==contributions]
    choices = [investiment]
    joel_performance['profit_total'] = np.select(conditions, choices, default=joel_performance['profit_total'])
    

In [45]:
joel_performance

,contributions,profit_total
0,1,10991.77
1,2,18868.28
2,3,28020.58
3,4,39934.46
4,5,51491.92
5,6,68296.89
6,7,72679.74
7,8,92025.41
8,9,106813.14
9,10,145217.24


In [47]:
date_detail_joel.to_csv('date_detail_joel.csv')

# 5) Graham Formula

In [48]:
df_graham=df_results_price_clean.copy()

In [49]:
df_graham=create_contributions(df_graham) 

In [50]:
df_graham['bvps']=df_graham['bvps'].astype(float)

In [51]:
df_graham=df_graham[df_graham['net_equity']>0]
df_graham=df_graham[df_graham['eps']>0].reset_index()

In [52]:
df_graham['vi']=(22.5*df_graham['eps']*df_graham['bvps'] )**(1/2)

In [53]:
df_graham['vi_percent']=df_graham['close']/df_graham['vi']-1

In [54]:
df_graham=df_graham.drop(columns=['index'])
df_graham=df_graham.drop(columns=['level_0'])

In [55]:
# Calculate grahan performance

In [56]:
grahan_performance = pd.DataFrame({'contributions': range(1,26), 'profit_total':range(1,26)})
date_detail_grahan= pd.DataFrame()
companies_keeped = pd.DataFrame({'company': list('a')})

investiment_month=10000
investiment=0
top=10



In [58]:
for contributions in range(26):
    investiment=investiment+investiment_month
    investiment_per_share=investiment/10

    df_graham_selected=df_graham[(df_graham['contributions']==contributions)].reset_index(drop=True)
    df_graham_selected=scoring(df=df_graham_selected, metric='vi_percent', ascend=True)
    df_keeped= df_graham_selected[df_graham_selected['company'].isin(companies_keeped['company'])]

    df_graham_selected_clean = df_graham_selected[~df_graham_selected['company'].isin(companies_keeped['company'])].reset_index(drop=True)
    df_new = df_graham_selected_clean[df_graham_selected_clean.index.isin(range(top))]
    
    df_graham_selected_1=pd.concat([df_new, df_keeped]).reset_index(drop=True)
    # profit 
    df_graham_selected_2=pd.merge(df_graham_selected_1,date_quote_end_3,how='left',left_on=['company','contributions'] , 
                                  right_on=['company','contributions'])
    df_graham_selected_2['no_shares']=investiment_per_share/df_graham_selected_2['close']
    df_graham_selected_2['profit']=df_graham_selected_2['no_shares']*(df_graham_selected_2['close_end'])

    investiment=df_graham_selected_2['profit'].sum() 
    
    
    # selecet the 10 TOP companies 
    date_detail_grahan= pd.concat([date_detail_grahan, df_graham_selected_2])

    # Condition
    conditions = [
        grahan_performance['contributions']==contributions]
    choices = [investiment]
    grahan_performance['profit_total'] = np.select(conditions, choices, default=grahan_performance['profit_total'])

    contributions_next=contributions+1
    df_graham_next=df_graham[df_graham['contributions']==contributions_next].reset_index()


    df_graham_next['contributions']=df_graham_next['contributions']-1
    df_graham_next=df_graham_next.rename(columns={'vi':'vi_end', 'vi_percent':'vi_percent_end'}) 
    df_graham_next=df_graham_next[['company', 'contributions', 'vi_end', 'vi_percent_end']]

    df_graham_selected_next=pd.merge(df_graham_selected_2,df_graham_next,how='left',left_on=['company','contributions'], 
                                     right_on=['company','contributions'])

    df_graham_selected_next_1=df_graham_selected_next[df_graham_selected_next['vi_percent_end']<=-0.3]

    companies_keeped=df_graham_selected_next_1[['company']]

    top=10-len(df_graham_selected_next_1)

In [70]:
date_detail_grahan.to_csv('date_detail_grahan.csv')

In [59]:
grahan_performance

,contributions,profit_total
0,1,9661.52
1,2,17500.34
2,3,29706.49
3,4,44754.30
4,5,64303.20
5,6,78592.49
6,7,100965.88
7,8,143643.53
8,9,153104.72
9,10,211497.32


# 6) Investment in Fixed income

## 6.1) Selic

In [66]:
meta_selic= pd.read_csv("meta_selic.csv", sep=";")

In [67]:
def change_date_format(df):
    df['date_month']=df['date'].str[3:5]
    df['date_year']=df['date'].str[6:10]
    df['date']=df['date'].str[6:10]+df['date'].str[3:5]+df['date'].str[0:2]
    df['date_month']=df['date_month'].astype(int)
    df['date_year']=df['date_year'].astype(int)
    
    return df

In [68]:
def annual_to_monthly_tax(df, annual_tax):
    df[annual_tax]=df[annual_tax].astype(float)
    df['monthly_tax']= ( (1 + df[annual_tax]/100)**(1/12) - 1)*100

    df_2=change_date_format(df)
    
    return df_2

In [69]:
meta_selic=annual_to_monthly_tax(meta_selic, 'meta_taxa_selic')

In [70]:
# contributions

In [71]:
def apply_contribution_fixed(df): 
    df=df.sort_values(by=['date'], ascending=False)
    df=create_contributions(df) 
    df=df.drop_duplicates(subset=['date_month','date_year'], keep = 'last')
    df=df[df['contributions']!=0].reset_index(drop=True)
    return df

In [72]:
meta_selic_2=apply_contribution_fixed(meta_selic)

In [73]:
def calculate_compound_interest(df):
    investiment_month=10000
    investiment=0
    df_final=pd.DataFrame()
    
    for contributions in range(26):
        contributions=contributions+1
        investiment=investiment+investiment_month
        df_month=df[df['contributions']==contributions].reset_index(drop=True)
        df_month['pnl']=investiment*(1+df_month['monthly_tax']/100)**3
        investiment=df_month['pnl'].values
        df_final = pd.concat([df_final, df_month])
    return df_final

In [74]:
meta_selic_final=calculate_compound_interest(meta_selic_2, )

In [75]:
meta_selic_final

,date,meta_taxa_selic,monthly_tax,date_month,date_year,contributions,pnl
0,20150301,12.25,0.97,3,2015,1,10293.11
0,20150601,13.25,1.04,6,2015,2,20934.29
0,20150901,14.25,1.12,9,2015,3,31981.89
0,20151201,14.25,1.12,12,2015,4,43403.63
0,20160301,14.25,1.12,3,2016,5,55212.17
0,20160601,14.25,1.12,6,2016,6,67420.62
0,20160901,14.25,1.12,9,2016,7,80042.51
0,20161201,13.75,1.08,12,2016,8,92989.82
0,20170301,12.25,0.97,3,2017,9,106008.55
0,20170601,10.25,0.82,6,2017,10,118873.39


# 6.2) ipca

In [76]:
ipca= pd.read_csv("ipca_historic.csv", sep=";")

In [77]:
ipca_1=annual_to_monthly_tax(ipca.copy(), 'ipca_12m')

In [78]:
ipca_2=apply_contribution_fixed(ipca_1)

In [79]:
ipca_final=calculate_compound_interest(ipca_2)

In [80]:
ipca_final

,date,ipca_12m,monthly_tax,date_month,date_year,contributions,pnl
0,20150301,8.13,0.65,3,2015,1,10197.33
0,20150601,8.89,0.71,6,2015,2,20631.98
0,20150901,9.49,0.76,9,2015,3,31334.21
0,20151201,10.67,0.85,12,2015,4,42395.24
0,20160301,9.39,0.75,3,2016,5,53584.14
0,20160601,8.84,0.71,6,2016,6,64945.03
0,20160901,8.48,0.68,9,2016,7,76485.70
0,20161201,6.29,0.51,12,2016,8,87814.74
0,20170301,4.57,0.37,3,2017,9,98913.62
0,20170601,3.00,0.25,6,2017,10,109721.44


# 6.3) IPCA + 5 % 

In [81]:
ipca_5=ipca.copy()

In [82]:
ipca_5['ipca_5_12m']=ipca_5['ipca_12m']+5

In [83]:
ipca_5_1=annual_to_monthly_tax(ipca_5, 'ipca_5_12m')

In [84]:
ipca_5_2=apply_contribution_fixed(ipca_5_1)

In [85]:
ipca_5_final=calculate_compound_interest(ipca_5_2)

In [86]:
ipca_5_final

,date,ipca_12m,ipca_5_12m,monthly_tax,date_month,date_year,contributions,pnl
0,20150301,8.13,13.13,1.03,3,2015,1,10313.22
0,20150601,8.89,13.89,1.09,6,2015,2,20984.58
0,20150901,9.49,14.49,1.13,9,2015,3,32050.70
0,20151201,10.67,15.67,1.22,12,2015,4,43609.23
0,20160301,9.39,14.39,1.13,3,2016,5,55441.70
0,20160601,8.84,13.84,1.09,6,2016,6,67597.14
0,20160901,8.48,13.48,1.06,9,2016,7,80089.49
0,20161201,6.29,11.29,0.90,12,2016,8,92531.20
0,20170301,4.57,9.57,0.76,3,2017,9,104900.83
0,20170601,3.00,8.00,0.64,6,2017,10,117132.96


# 7) Index

In [87]:
def calculate_stock_profit(df):
    investiment_month=10000
    investiment=0
    df_final=pd.DataFrame()
    
    for contributions in range(26):
        contributions=contributions+1
        investiment=investiment+investiment_month
        df_month=df[df['contributions']==contributions].reset_index(drop=True)
        df_month['pnl']=investiment*(1+df_month['monthly_tax']/100)
        investiment=df_month['pnl'].values
        df_final = pd.concat([df_final, df_month])
    return df_final

## 7.1) Bova11

In [89]:
index= pd.read_csv("index_bova.csv", sep=";")

In [90]:
index_2=change_date_format(index)

In [91]:
index_c=apply_contribution_fixed(index_2)

In [92]:
# Create a monthly tax 

In [93]:
index_result=index_c[[ 'contributions', 'close']].reset_index(drop=True)

In [94]:
index_result['contributions']=index_result['contributions']-1

In [95]:
index_result=index_result.rename(columns={"close": "close_end"})

In [96]:
index_result_2=pd.merge(index_c,index_result,how='left',left_on=['contributions'],right_on=['contributions'])

In [97]:
index_result_2['monthly_tax']=((index_result_2['close_end']-index_result_2['close'] )/index_result_2['close']  )*100

In [98]:
index_result_bova=calculate_stock_profit(index_result_2)

In [99]:
index_result_bova

,date,open,high,low,close,adj close,volume,company,date_month,date_year,contributions,close_end,monthly_tax,pnl
0,20150302,49.99,50.10,49.34,49.59,49.59,1584990,BOVA11,3,2015,1,51.46,3.77,10377.09
0,20150601,51.18,51.76,51.14,51.46,51.46,3826350,BOVA11,6,2015,2,44.17,-14.17,17490.40
0,20150901,44.29,44.67,43.84,44.17,44.17,4702010,BOVA11,9,2015,3,43.53,-1.45,27092.08
0,20151201,43.86,44.08,43.41,43.53,43.53,2462470,BOVA11,12,2015,4,42.75,-1.79,36427.44
0,20160301,42.01,43.12,41.87,42.75,42.75,2934210,BOVA11,3,2016,5,47.48,11.06,51564.33
0,20160601,47.02,47.62,46.76,47.48,47.48,1778110,BOVA11,6,2016,6,56.25,18.47,72935.83
0,20160901,56.55,56.61,55.85,56.25,56.25,3471900,BOVA11,9,2016,7,57.80,2.76,85221.17
0,20161201,57.80,57.80,57.80,57.80,57.80,7064250,BOVA11,12,2016,8,64.83,12.16,106802.58
0,20170301,65.12,65.69,64.69,64.83,64.83,1175660,BOVA11,3,2017,9,60.26,-7.05,108568.92
0,20170601,60.90,61.21,60.12,60.26,60.26,1564630,BOVA11,6,2017,10,69.51,15.35,136769.43


## 7.2) IVVB11

In [101]:
index= pd.read_csv("index_ivvb.csv", sep=";")

In [102]:
index_2=change_date_format(index)

In [103]:
index_c=apply_contribution_fixed(index_2)

In [104]:
index_result=index_c[[ 'contributions', 'close']].reset_index(drop=True)

In [105]:
index_result['contributions']=index_result['contributions']-1

In [106]:
index_result=index_result.rename(columns={"close": "close_end"})

In [107]:
index_result_2=pd.merge(index_c,index_result,how='left',left_on=['contributions'],right_on=['contributions'])

In [109]:
index_result_2['monthly_tax']=((index_result_2['close_end']-index_result_2['close'] )/index_result_2['close']  )*100

In [110]:
index_result_ivvb=calculate_stock_profit(index_result_2)

In [111]:
index_result_ivvb

,date,close,company,date_month,date_year,contributions,close_end,monthly_tax,pnl
0,20150302,62.23,IVVB11,3,2015,1,68.21,9.61,10960.95
0,20150601,68.21,IVVB11,6,2015,2,72.14,5.76,22168.64
0,20150901,72.14,IVVB11,9,2015,3,82.89,14.90,36962.28
0,20151201,82.89,IVVB11,12,2015,4,79.98,-3.51,45313.58
0,20160301,79.98,IVVB11,3,2016,5,77.77,-2.76,53785.16
0,20160601,77.77,IVVB11,6,2016,6,72.93,-6.22,59815.51
0,20160901,72.93,IVVB11,9,2016,7,78.74,7.97,75377.39
0,20161201,78.74,IVVB11,12,2016,8,77.15,-2.02,83653.36
0,20170301,77.15,IVVB11,3,2017,9,82.25,6.61,99844.32
0,20170601,82.25,IVVB11,6,2017,10,81.59,-0.80,108962.89


# ---